# DB 연결
### CRUD
- (Commit 필요 X, Trigger 작동 X) Select
- (Commit 필요 O, Trigger 작동 O) Insert, Update, Delete
- 2 개의 Cursor와 Connection 존재 => System자원 및 네트워크 사용.
- 
- 반드시 close() 해줘야만 함.

## Select

In [ ]:
import pymysql
con = pymysql.connect(host = 'localhost', port=3307, user='root', passwd='0000',
                      db='sungjuk_test', charset='utf8', autocommit = True)


# SQL Connection 오픈 상태로 작업을 수행 중 에러 발생 시 Connection 닫히지 않는 문제 발생.
# 이 상태가 반복되면 Connection Leak 현상 발생. 새로운 Connection 오픈 불가능.
# 해당 문제 해결을 위해 try & finally 사용.
# finally에서 Connection Close() 수행.
try :
    with con.cursor() as cur:
        cur.execute("select * from student")  # execute, executemany, callproc => Recordset
        rows = cur.fetchall()                 # fetchone, fetchmany, fetchall
        desc = cur.description
        
        # f 문자열, % 문자열, Format 문자열
        print(f'{desc[0][0]:<8} {desc[1][0]:<15} {desc[2][0]:>10} {desc[3][0]:>10}')
        
        for row in rows:
            print(f'{row[0]}, {row[1]}, {row[2]}')    # index를 통해서 => 기본커서 tuple 커서를 사용
        
        print(f'쿼리가 영향을 미친 행은 {cur.rowcount} 개의 행')

finally:
    con.close()

## Insert

In [ ]:
import pymysql
con = pymysql.connect(host = 'localhost', port=3307, user='root', passwd='0000',
                      db='sungjuk_test', charset='utf8', autocommit = True)

person = ('한국이', 100, 100, 100, 'CH00000001')

try :
    with con.cursor() as cur:
        
        # Placeholder
        # 문자열을 %s로 지정하면 int던 date던 타입이 어찌되었건 알아서 입력됨.
        cur.execute("insert into student(name, kor, mat, eng, schoolcode) values(%s, %s, %s, %s, %s)",
                   (person[0], person[1], person[2], person[3], person[4]))
        
        con.commit()
        print('새로운 학생이 등록되었습니다.')

finally:
    con.close()

## Create

In [ ]:
import pymysql
con = pymysql.connect(host = 'localhost', port=3307, user='root', passwd='0000',
                     db='sungjuk_test', charset='utf8', autocommit = True,
                     cursorclass = pymysql.cursors.DictCursor)

cursor = con.cursor()
cursor.execute("""
CREATE TABLE `pet` (
`id` INT(11) NOT NULL AUTO_INCREMENT,
`name` VARCHAR(20) NOT NULL COLLATE 'utf8_general_ci',
`owner` VARCHAR(20) NULL DEFAULT NULL COLLATE 'utf8_general_ci',
`species` VARCHAR(20) NULL DEFAULT NULL COLLATE 'utf8_general_ci',
`sex` CHAR(1) NULL DEFAULT NULL COLLATE 'utf8_general_ci',
`birth` DATE NULL DEFAULT NULL,
`death` DATE NULL DEFAULT NULL,
PRIMARY KEY (`id`) USING BTREE
)
COLLATE='utf8_general_ci'
ENGINE=InnoDB
;""")
cursor.close()
con.close()

In [5]:
import pymysql
con = pymysql.connect(host = 'localhost', port=3307, user='root', passwd='0000',
                     db='sungjuk_test', charset='utf8', autocommit = True,
                     cursorclass = pymysql.cursors.DictCursor)

try :
    with con.cursor() as cur:
        cur.executemany('insert into pet values (%s, %s, %s, %s, %s, %s, %s)',
                   [(1, '인천시', '민주', 'CAT', 'F', '2001-02-04', None),
                    (2, '대구시', '자유', 'CAT', 'F', '2010-03-17', None),
                    (3, '대전시', '민주', 'DOG', 'F', '2010-05-03', None),
                    (4, '광주시', '민주', 'DOG', 'M', '2015-08-27', None),
                    (5, '부산시', '자유', 'DOG', 'M', '2017-08-31', '2018-04-29')
                   ])
        con.commit()
        print(f'총 {cur.rowcount} 개의 행이 등록되었습니다.')
        
finally:
    con.close()

총 5 개의 행이 등록되었습니다.


In [8]:
import pymysql
con = pymysql.connect(host = 'localhost', port=3307, user='root', passwd='0000',
                     db='sungjuk_test', charset='utf8', autocommit = True)

try:
    with con.cursor() as cur:
        cur.execute("select * from pet")

        desc = cur.description
        print(f'{desc[0][0]}\t{desc[1][0]}\t{desc[2][0]}\t{desc[3][0]}\t{desc[4][0]}\t{desc[5][0]}\t\t{desc[6][0]}')

        rows = cur.fetchall()
        for row in rows:
            print(f'{row[0]}\t{row[1]}\t{row[2]}\t{row[3]}\t{row[4]}\t{row[5]}\t{row[6]}')
        
finally:
    con.close()

id	name	owner	species	sex	birth		death
1	인천시	민주	CAT	F	2001-02-04	None
2	대구시	자유	CAT	F	2010-03-17	None
3	대전시	민주	DOG	F	2010-05-03	None
4	광주시	민주	DOG	M	2015-08-27	None
5	부산시	자유	DOG	M	2017-08-31	2018-04-29


##### Pet에 Select Procedure 생성하고 클라이언트에서 출력

In [19]:
import pymysql
con = pymysql.connect(host = 'localhost', port=3307, user='root', passwd='0000',
                     db='sungjuk_test', charset='utf8', autocommit = True,
                     cursorclass = pymysql.cursors.DictCursor)

try:
    with con.cursor() as cur:
        cur.callproc('pet_select')  # 레코드셀 생성.
        if (cur.rowcount):
            for i in range(cur.rowcount):
                print(cur.fetchone())

finally:
    con.close()

{'id': 1, 'name': '인천시', 'owner': '민주', 'species': 'CAT', 'sex': 'F', 'birth': datetime.date(2001, 2, 4), 'death': None}
{'id': 2, 'name': '대구시', 'owner': '자유', 'species': 'CAT', 'sex': 'F', 'birth': datetime.date(2010, 3, 17), 'death': None}
{'id': 3, 'name': '대전시', 'owner': '민주', 'species': 'DOG', 'sex': 'F', 'birth': datetime.date(2010, 5, 3), 'death': None}
{'id': 4, 'name': '광주시', 'owner': '민주', 'species': 'DOG', 'sex': 'M', 'birth': datetime.date(2015, 8, 27), 'death': None}
{'id': 5, 'name': '인천시', 'owner': '자유', 'species': 'DOG', 'sex': 'M', 'birth': datetime.date(2017, 8, 31), 'death': datetime.date(2018, 4, 29)}


##### Pet에 Insert Procedure로 데이터 생성하고 클라이언트에서 출력

In [15]:
import pymysql
con = pymysql.connect(host = 'localhost', port=3307, user='root', passwd='0000',
                     db='sungjuk_test', charset='utf8', autocommit = True,
                     cursorclass = pymysql.cursors.DictCursor)

data = ('부산시', '민주', 'CAT', 'M', '2001-02-04', '2019-12-31', 0)

try:
    with con.cursor() as cur:
        cur.callproc('pet_insert', data)
        print({cur.rowcount},' 행이 삽입되었습니다.')

finally:
    con.close()

{1}  행이 삽입되었습니다.


##### Pet에 Update Procedure로 데이터를 변경하고 클라이언트에서 출력

In [14]:
import pymysql
con = pymysql.connect(host = 'localhost', port=3307, user='root', passwd='0000',
                     db='sungjuk_test', charset='utf8', autocommit = True,
                     cursorclass = pymysql.cursors.DictCursor)

data = ('부산시', '인천시', 0)

try:
    with con.cursor() as cur:
        cur.callproc('pet_update_name', data)
        cur.execute('select @_pet_update_name_2')     # 사용자 정의 변수(@_). 매개변수의 2번째(인덱스 순서, result)를 찾기.
        result = cur.fetchone()
        print('result = ', result['@_pet_update_name_2'])
        if result['@_pet_update_name_2'] == 2:
            print('수정할 이름이 없습니다. \n')
        elif result['@_pet_update_name_2'] == -1:
            print('실행 중 에러가 발생하였습니다. \n')
        else:
            print({cur.rowcount},' 행이 수정되었습니다.')

finally:
    con.close()

result =  2
수정할 이름이 없습니다. 



##### Pet에 Delete Procedure로 데이터를 삭제하고 클라이언트 출력

In [29]:
import pymysql
con = pymysql.connect(host = 'localhost', port=3307, user='root', passwd='0000',
                     db='sungjuk_test', charset='utf8', autocommit = True,
                     cursorclass = pymysql.cursors.DictCursor)

data = ('6' , 0)

try:
    
    with con.cursor() as cur:
        cur.callproc('pet_select')
        if (cur.rowcount):
            print('[쿼리 수행 전 데이터 출력]')
            for i in range(cur.rowcount):
                print(cur.fetchone())
    
    
    with con.cursor() as cur:
        cur.callproc('pet_delete', data)
        cur.execute('select @_pet_delete_1')
        result = cur.fetchone()
        if result['@_pet_delete_1'] == 2:
            print('\n삭제할 데이터가 존재하지 않습니다. \n')
        elif result['@_pet_delete_1'] == -1:
            print('\n실행 중 에러가 발생하였습니다. \n')
        else:
            print('\n',{cur.rowcount},'행이 삭제되었습니다. \n')
         
        
    with con.cursor() as cur:
        cur.callproc('pet_select')
        if (cur.rowcount):
            print('[쿼리 수행 후 데이터 출력]')
            for i in range(cur.rowcount):
                print(cur.fetchone())
            
finally:
    con.close()

[쿼리 수행 전 데이터 출력]
{'id': 1, 'name': '인천시', 'owner': '민주', 'species': 'CAT', 'sex': 'F', 'birth': datetime.date(2001, 2, 4), 'death': None}
{'id': 2, 'name': '대구시', 'owner': '자유', 'species': 'CAT', 'sex': 'F', 'birth': datetime.date(2010, 3, 17), 'death': None}
{'id': 3, 'name': '대전시', 'owner': '민주', 'species': 'DOG', 'sex': 'F', 'birth': datetime.date(2010, 5, 3), 'death': None}
{'id': 4, 'name': '광주시', 'owner': '민주', 'species': 'DOG', 'sex': 'M', 'birth': datetime.date(2015, 8, 27), 'death': None}
{'id': 5, 'name': '인천시', 'owner': '자유', 'species': 'DOG', 'sex': 'M', 'birth': datetime.date(2017, 8, 31), 'death': datetime.date(2018, 4, 29)}

삭제할 데이터가 존재하지 않습니다. 

[쿼리 수행 후 데이터 출력]
{'id': 1, 'name': '인천시', 'owner': '민주', 'species': 'CAT', 'sex': 'F', 'birth': datetime.date(2001, 2, 4), 'death': None}
{'id': 2, 'name': '대구시', 'owner': '자유', 'species': 'CAT', 'sex': 'F', 'birth': datetime.date(2010, 3, 17), 'death': None}
{'id': 3, 'name': '대전시', 'owner': '민주', 'species': 'DOG', 'sex': 'F', 

## Tuple 을 Dict로 전환

In [42]:
import pymysql
con = pymysql.connect(host = 'localhost', port=3307, user='root', passwd='0000',
                     db='sungjuk_test', charset='utf8', autocommit = True,
                     cursorclass = pymysql.cursors.DictCursor)

try:
    with con.cursor() as cursor:
        sql = 'insert into users(email, password) values(%s, %s)'
        cursor.execute(sql, ('kpc01@kakao.com', '0001'))
        cursor.execute(sql, ('kpc02@kakao.com', '0002'))
        con.commit()
    with con.cursor() as cursor:
        sql = 'select * from users'
        cursor.execute(sql)
        result = cursor.fetchall()
        for dic in result:
            for key in dic.keys():
                print(key,'',end='')

finally:
    con.close()
        

dict_keys(['id', 'email', 'password'])
id email password id email password id email password id email password id email password id email password id email password id email password id email password id email password id email password id email password id email password id email password 

# 예제
원격으로 pymysql을 이용하여 자신의 id와 password를 입력, 출력 하시오

In [ ]:
import pymysql
con = pymysql.connect(host = 'localhost', port=3307, user='root', passwd='0000',
                      db='sungjuk_test', charset='utf8', autocommit = True)

idpw = (00000, 'kpc@kakao.com', '0000')

try :
    with con.cursor() as cur:
        
        cur.execute("insert into users(id, email, password) values(%s, %s, %s)", (idpw[0], idpw[1], idpw[2]))
        print('새로운 학생이 등록되었습니다. \n')
        
        cur.execute("select * from users")
        rows = cur.fetchall()
        desc = cur.description
        
        print(f'{desc[0][0]}\t{desc[1][0]}\t{desc[2][0]}')
        for row in rows:
            print(f'{row[0]}\t{row[1]}\t{row[2]}')
        
        print(f'\n등록된 쿼리는 총 {cur.rowcount} 개의 행')
        
finally:
    con.close()

# 문제
- Student Class에 CRUD 생성.
- DB Class 생성하여 Student Class의 CRUD와 연결.
- Management Class 생성하여 메뉴 생성.

In [23]:
class Student:
    studCount = 0
    students = {}
    def __init__(self, name, kor, mat, eng):
        self.name = name
        self.kor = int(kor)
        self.mat = int(mat)
        self.eng = int(eng)
        Student.studCount += 1
        
    def studentInsert(students):
        print(" ")
        name = input("이름 입력 : ")
        kor = int(input("국어성적 입력 : "))
        mat = int(input("수학성적 입력 : "))
        eng = int(input("영어성적 입력 : "))
        cod = input("학교 코드 입력 : ")
        students['name'] = name
        students['kor'] = kor
        students['mat'] = mat
        students['eng'] = eng
        students['code'] = cod
        return students
    
    def studentDelete():
        num = int(input("\n삭제할 데이터의 번호를 입력하세요 : "))
        return num
    
    def studentSelect():
        return null
    
    def studentUpdate(students):
        id = int(input("\n수정하고 싶은 row_Number를 입력하세요 : "))
        students['id'] = id
        while 1:
            print("\n[이름: 1 \t 국어성적: 2 \t 수학성적: 3 \t 영어성적: 4 \t 실행: 9]")
            menu = int(input("수정할 내용을 선택하세요 => "))
            if menu == 1:
                name = input("수정할 이름 입력 : ")
                students['name'] = name
            elif menu == 2:
                kor = int(input("수정할 국어성적 입력 : "))
                students['kor'] = kor
            elif menu == 3:
                mat = int(input("수정할 수학성적 입력 : "))
                students['mat'] = mat
            elif menu == 4:
                eng = int(input("수정할 영어성적 입력 : "))
                students['eng'] = eng
            elif menu == 9:
                break     
        return students

In [35]:
import pymysql

class DB:
    def DB_Insert():
        
        con = pymysql.connect(host = 'localhost', port=3307, user='root', passwd='0000',
                      db='sungjuk_test', charset='utf8', autocommit = True,
                      cursorclass = pymysql.cursors.DictCursor)

        Dic = {}
        insertDic = Student.studentInsert(Dic)
        insertData = (insertDic['name'], insertDic['kor'], insertDic['mat'], insertDic['eng'], insertDic['code'], 0)

        try :
            with con.cursor() as cur:
                cur.callproc('student_insert', insertData)
                cur.execute('select @_student_insert_5')
                result = cur.fetchone()
                if result['@_student_insert_5'] == -1:
                    print('\n실행 중 에러가 발생하였습니다. \n')
                else :
                    print('\n새로운 학생이 등록되었습니다. \n')
                
        finally:
            con.close()
            
            
            
            
            
    def DB_Delete():
        
        con = pymysql.connect(host = 'localhost', port=3307, user='root', passwd='0000',
                      db='sungjuk_test', charset='utf8', autocommit = True,
                      cursorclass = pymysql.cursors.DictCursor)
        
        deleteNum = Student.studentDelete()
        deleteData = (deleteNum, 0)
        
        try :
            with con.cursor() as cur :
                cur.callproc('student_delete', deleteData)      
                cur.execute('select @_student_delete_1')
                result = cur.fetchone()
                if result['@_student_delete_1'] == 2:
                    print('\n삭제할 데이터가 존재하지 않습니다. \n')
                elif result['@_student_delete_1'] == -1:
                    print('\n실행 중 에러가 발생하였습니다. \n')
                else:
                    print('\n',{cur.rowcount},'행이 삭제되었습니다. \n')
                    
#                 cur.execute('delete from student where bunho = %s', deleteNum)
#                 row = cur.rowcount
#                 if row == 1:
#                     print(f'\n{row}행이 삭제되었습니다. \n')
#                 else :
#                     print('\n삭제할 데이터가 존재하지 않습니다. \n')  
                    
        finally :
            con.close()
            
            
            
            
            
    def DB_Select():
        
        con = pymysql.connect(host = 'localhost', port=3307, user='root', passwd='0000',
                      db='sungjuk_test', charset='utf8', autocommit = True)
        
        try :
            with con.cursor() as cur:
                cur.callproc('student_select')
                
                desc = cur.description
                print(f'\n{desc[0][0]}\t{desc[1][0]}\t{desc[2][0]}\t{desc[3][0]}\t{desc[4][0]}\t{desc[5][0]}\t{desc[6][0]}\t{desc[7][0]}\t{desc[8][0]}')

                rows = cur.fetchall()
                for row in rows:
                    print(f'{row[0]}\t{row[1]}\t{row[2]}\t{row[3]}\t{row[4]}\t{row[5]}\t{row[6]}\t{row[7]}\t{row[8]}')
                    
        finally :
            con.close()
            
            
            
            
            
    def DB_Update():
        
        con = pymysql.connect(host = 'localhost', port=3307, user='root', passwd='0000',
                      db='sungjuk_test', charset='utf8', autocommit = True,
                      cursorclass = pymysql.cursors.DictCursor)
        
        Dic = {}
        Data = {}
        updateDic = Student.studentUpdate(Dic)
        
        try :
            with con.cursor() as cur :
                
                cur.execute("select * from student where bunho = %s",updateDic['id'])
                res = cur.fetchall()
                
                if not updateDic.get('name'):
                    Data.setdefault('name', res[0]['name'])
                else :
                    Data.setdefault('name', updateDic['name'])
                    
                if not updateDic.get('kor'):
                    Data.setdefault('kor', res[0]['kor'])
                else :
                    Data.setdefault('kor', updateDic['kor'])
                    
                if not updateDic.get('mat'):
                    Data.setdefault('mat', res[0]['mat'])
                else :
                    Data.setdefault('mat', updateDic['mat'])
                    
                if not updateDic.get('eng'):
                    Data.setdefault('eng', res[0]['eng'])
                else :
                    Data.setdefault('eng', updateDic['eng'])
                    
            with con.cursor() as cur :
                cur.callproc('student_update',
                            (updateDic['id'], Data['name'], Data['kor'], Data['mat'], Data['eng'], 0))
                cur.execute('select @_student_update_5')
                result = cur.fetchone()
                if result['@_student_update_5'] == 2:
                    print('\n해당 id에 할당된 데이터가 없습니다. \n')
                elif result['@_student_update_5'] == -1:
                    print('\n실행 중 에러가 발생하였습니다. \n')
                else:
                    print('\n',{cur.rowcount},'행이 업데이트 되었습니다. \n')
                
                
        finally:
            con.close()

In [33]:
class Management:
    def start():
        while 1:
            choice = input("\n[ 1: 입력 \t 2: 출력 \t 3: 수정 \t 4: 삭제 \t 9: 종료 ] => ")
            if choice == "1":
                print('\n\n\n데이터 입력을 시작합니다. \n')
                DB.DB_Insert()
            elif choice == "2":
                print('\n\n\n데이터를 출력합니다. \n')
                DB.DB_Select()
            elif choice == "3":
                print('\n\n\n데이터 수정을 시작합니다. \n')
                DB.DB_Update()
            elif choice == "4":
                print('\n\n\n 데이터 삭제를 시작합니다. \n')
                DB.DB_Delete()
            elif choice == "9":
                print("\n\n\n프로그램을 종료합니다. \n")
                break

In [36]:
Management.start()


[ 1: 입력 	 2: 출력 	 3: 수정 	 4: 삭제 	 9: 종료 ] => 1



데이터 입력을 시작합니다. 

 
이름 입력 : Min
국어성적 입력 : 50
수학성적 입력 : 60
영어성적 입력 : 70
학교 코드 입력 : 4

새로운 학생이 등록되었습니다. 


[ 1: 입력 	 2: 출력 	 3: 수정 	 4: 삭제 	 9: 종료 ] => 2



데이터를 출력합니다. 


bunho	name	kor	mat	eng	total	average	grade	schoolcode
1	김만덕	100	99	99	298	99.33	A	CH00000001
2	고려인	100	99	99	298	99.33	A	CH00000001
3	종로구	100	80	70	250	83.33	B	IC00000001
4	전공인	80	80	80	240	80.0	B	SE00000001
5	전공이	80	80	80	240	80.0	B	SE00000001
6	만세	74	64	92	230	76.67	C	None
7	한국이	100	100	100	300	100.0	A	CH00000001
16	Test	60	80	42	182	60.67	D	None
17	Min	50	60	70	180	60.0	D	None

[ 1: 입력 	 2: 출력 	 3: 수정 	 4: 삭제 	 9: 종료 ] => 9



프로그램을 종료합니다. 

